In [1]:
import os
import time
import importlib
import transformers
from datasets import load_dataset, load_from_disk
import torch
import re
import jinja2

import rpbuild as rp
import rpbuild.char
import rpbuild.data
import rpbuild.writer
import rpbuild.director
import rpbuild.roleplay

from rpbuild import load_template

# Trigger dynamic reload of module -- for editing without restarting the kernel
importlib.reload(rp.director)

<module 'rpbuild.director' from '/home/dinalt/rust/ai_development/roleplay_build/rpbuild/director.py'>

### Load Resources
Load dataset and model for testing...

In [3]:
from transformers import BitsAndBytesConfig

# The location of the input dataset
dataset_id = "dinalt/roleplay_build"

# Load dataset
dataset = load_dataset(dataset_id)["train"]
print(dataset)

# Lineage of "fhai50032/RolePlayLake-7B"
# https://huggingface.co/fhai50032/RolePlayLake-7B
#     https://huggingface.co/SanjiWatsuki/Silicon-Maid-7B
#         https://huggingface.co/xDAN-AI/xDAN-L1-Chat-RL-v1
#     https://huggingface.co/senseable/WestLake-7B-v2
#

# Where are models stored?
models_dir = "/home/dinalt/ai_assets/models"

# Configure a model to use.
# The name of this model -- which will live in models_dir
#model_name = "fhai50032_RolePlayLake-7B" # AKA "fhai50032/RolePlayLake-7B"
#model_id = os.path.join(models_dir, model_name)

# Or... or load it from the hub / cache
model_id = "fhai50032/RolePlayLake-7B"

# Device to run model on; set to 0 (or device name), when not using quantization
device = None

# Load model with quantization
# Quantization is enabled by default, for those with low GPU memory.
# If you have enough memory, disabled it. It's faster and produces better output.

# See link for configuration options alternatives
# https://huggingface.co/docs/transformers/main/en/quantization

# Load model and tokenizer
causal_lm = rp.CausalLM(
    model_id,
    device=device,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    instruct_template=load_template("instruct/alpaca.jinja"),
    chat_template=load_template("models/original.jinja"),
    #device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
    )
)

Dataset({
    features: ['pairing_reason', 'plist', 'director_log', 'scenario', 'proxy', 'example_dialog', 'conversation', 'char_name', 'description', 'summary', 'preset', 'greeting'],
    num_rows: 2770
})
Tokenizer uses "right" padding; this may require moving it to "left" for batch generation.


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNor

### Examine Characters
Get a random character from the dataset and dump it

In [4]:
transformers.set_seed(42)
example_char = rp.data.random_char(dataset)
#example_char = dataset[0]
rp.data.dump_character_data(example_char)


char_name: Olivia Jones

summary: Olivia Jones is a dynamic and successful businesswoman who has made a name for herself in the competitive real estate industry. Her strategic mindset and unwavering determination have helped her achieve remarkable success in a male-dominated field.

preset: Midnight-Enigma

pairing_reason: The combination of Olivia's business acumen and expertise in real estate with Dr. Evelyn Nova's scientific knowledge and innovative approach could lead to a compelling story involving groundbreaking technologies or innovative business strategies.
==================================description===================================

Name: Olivia Jones
Human
35 years old
Tall stature, around 6 ft. (183 cm)
Sleek blonde hair, pulled back into a tight bun
Sharp hazel eyes
Dressed in professional attire, suits and blouses
Confident and assertive demeanor

Olivia Jones is a dynamic and successful businesswoman who has made a name for herself in the competitive real estate indu

## Create Test Characters

Get a random character (meta-data) and a random character to represent the user.

Instantiate a "Character" object from the meta-data.

In [7]:
transformers.set_seed(53)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
example_user = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
rp.data.print_char_summary(example_char, example_user)

Char: Lucas Frostbeard

Char Summary: A wise and experienced elder, Lucas Frostbeard shares his knowledge of Arctic survival and history with younger explorers, ensuring the preservation of traditions and customs.

Proxy User: Elara Nightwalker

Proxy User Summary: A rogue with a dark past, Elara Nightwalker possesses the ability to manipulate shadows and darkness. She uses her skills to navigate the underground world undetected, making her a formidable infiltrator and assassin.



### Text Rendering
#### System Prompt
Message rendering for characters in abstracted through the TemplateConfig() class.

In this example, we are passing the raw character meta data to the renderer. Note that template substitutions have not been performed on the dialog examples yet.

They will become concrete after we attached the metadata to a Character object.

In [8]:
# Create a default template config
t_config = rp.char.TemplateConfig(chat_template=causal_lm.chat_template)

# Render the default system prompt
print(t_config.render_system(example_char, example_user))

Name: Lucas Frostbeard
Arctic Hare, with a thick coat of snowy white fur and piercing blue eyes that seem to glow in the winter sun.
Sprinkled with grey hairs around muzzle and ears, giving him an air of wisdom and experience.
6 ft tall, broad shoulders, and strong build from a lifetime spent living in harsh Arctic conditions.

Over 70 winters old, with deep lines etched on his face and hands that bore the marks of countless battles against the elements.

Lucas Frostbeard is a revered elder among the Arctic community, serving as a mentor and guide to young explorers venturing into the unforgiving landscape. He possesses vast knowledge of Arctic survival techniques, passed down through generations, and is dedicated to sharing these valuable skills with future generations.

His calm demeanor exudes patience and understanding, making him a trusted confidant and advisor. Despite his age, he remains active and engaged in daily life, eagerly participating in hunting expeditions and communal 

#### Character Instructions
The last dialog message from a character's counter-part can contain instrucitons after the dialog. We will be using these to remind the character who they are, explain how to format dialog, and provide custom taylored instructions for the context.

You will not need to fill these out, but it's worth mentioning that the formatting is controlled by a configurable jinja template. In this case, we are using the default template.

In [9]:
print(
    t_config.render_instruct(
        content=example_char.name + ": Hi there, nice to meet you!",
        char=example_char,
        user=example_user,
        instruction=example_user.name + ", introduce yourself and describe the surrondings."
    )
)

Lucas Frostbeard: Hi there, nice to meet you!
### Instruction:
Write the next response for Lucas Frostbeard.
Keep your response short.
[Lucas Frostbeard: wise, experienced, elderly Arctic Hare; thick snowy white fur, piercing blue eyes that glow in winter sun, grey hairs around muzzle and ears, 6 ft tall, broad shoulders, strong build, over 70 winters old, deep lines etched face, hands marked with battle scars; Revered elder, mentor, guide, deep knowledge of Arctic survival, shares skills, patient, understanding, calm demeanor, trusted confidant, advisor, active, engages in daily life, values traditions, preserves ancestral stories, tends garden, enjoys long walks; Genre: mythology, folklore; Tags: Arctic, elder, mentor, wisdom, survival, storytelling; Scenario: {{char}} asks Lucas for advice on navigating the harsh Arctic terrain. Lucas shares a story from his vast repertoire, imparting valuable knowledge and insights. ]

Elara Nightwalker, introduce yourself and describe the surrondi

#### Dialog Rendering

We abstract the model specific formatting via the chat_template. Ideally, we will be able to use the one provided with the model (attached to "tokenizer.chat_template")

In this example, we create a TemplateConfig with a "canned" ChatML template, create some sample dialog, and use the object to render the conversation.

The TemplateConfig class is highly configurable. Take a look at the constructor when you have a chance.

In [10]:
# Create a template config with the ChatML template for dialog.
t_config = rp.char.TemplateConfig(chat_template=load_template("models/chatml.jinja"))

messages = [
    {
        "role": "user",
        "content": "Hi there, nice to meet you.",
    },
    {
        "role": "assistant",
        "content": "*looks annoyed*",
    },
]

print(
    t_config.render_conversation(
        messages=messages,
        add_generation_prompt=True,
    )
)


<|im_start|>user
Hi there, nice to meet you.<|im_end|>
<|im_start|>assistant
*looks annoyed*<|im_end|>
<|im_start|>assistant



### The Character Class

Next, we initialize the a new Character object with the metadata. We also use the default chat template.

The Character class is primarly responsible for managing dialog history and generation of responses.

In [11]:
transformers.set_seed(43)
char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    user_meta=example_user,
    template_config=rp.char.TemplateConfig(
        chat_template=causal_lm.chat_template,
    ),
)

Token indices sequence length is longer than the specified maximum sequence length for this model (951 > 255). Running this sequence through the model will result in indexing errors


### Show Character's System Prompt
The Character class uses the TemplateConfig() object to format the system prompt, but it has automatically substituted all of the names.

In [12]:
print(char.system_prompt())

Name: Lucas Frostbeard
Arctic Hare, with a thick coat of snowy white fur and piercing blue eyes that seem to glow in the winter sun.
Sprinkled with grey hairs around muzzle and ears, giving him an air of wisdom and experience.
6 ft tall, broad shoulders, and strong build from a lifetime spent living in harsh Arctic conditions.

Over 70 winters old, with deep lines etched on his face and hands that bore the marks of countless battles against the elements.

Lucas Frostbeard is a revered elder among the Arctic community, serving as a mentor and guide to young explorers venturing into the unforgiving landscape. He possesses vast knowledge of Arctic survival techniques, passed down through generations, and is dedicated to sharing these valuable skills with future generations.

His calm demeanor exudes patience and understanding, making him a trusted confidant and advisor. Despite his age, he remains active and engaged in daily life, eagerly participating in hunting expeditions and communal 

### Add Character's Greeting
The character metadata contains a pre-generated "greeting" message, which can be used to seed the conversation.

In [13]:
char.greet()
char.print_conversation()

------------------------------ system:system (951)------------------------------
Name: Lucas Frostbeard
Arctic Hare, with a thick coat of snowy white fur and piercing blue eyes that seem to glow in the winter sun.
Sprinkled with grey hairs around muzzle and ears, giving him an air of wisdom and experience.
6 ft tall, broad shoulders, and strong build from a lifetime spent living in harsh Arctic conditions.

Over 70 winters old, with deep lines etched on his face and hands that bore the marks of countless battles against the elements.

Lucas Frostbeard is a revered elder among the Arctic community, serving as a mentor and guide to young explorers venturing into the unforgiving landscape. He possesses vast knowledge of Arctic survival techniques, passed down through generations, and is dedicated to sharing these valuable skills with future generations.

His calm demeanor exudes patience and understanding, making him a trusted confidant and advisor. Despite his age, he remains active and 

### Insert User Dialog
Add user dialog to the history.

In [14]:
char.user_says("I'd love to learn about ancient Egypt.")

{'role': 'user',
 'content': "I'd love to learn about ancient Egypt.",
 'tokens': 11,
 'name': 'Elara Nightwalker'}

### Show Character's Chat Prompt
Again, the TemplateConfig() class will be used for rendering. We insert instructions at the end of the last "user" message, as we saw above.

We can insert a custom instruction to guide each response. This is normally reserved for the director role, but let's demonstrate the influence it has be instructing the actor to brush aside the out-of-context question.

In [15]:
instruction = example_char.name + ", look confused, then ignore the questions and talk about your favorite subject, cats, instead."
print(char.chat_prompt(instruction))

Name: Lucas Frostbeard
Arctic Hare, with a thick coat of snowy white fur and piercing blue eyes that seem to glow in the winter sun.
Sprinkled with grey hairs around muzzle and ears, giving him an air of wisdom and experience.
6 ft tall, broad shoulders, and strong build from a lifetime spent living in harsh Arctic conditions.

Over 70 winters old, with deep lines etched on his face and hands that bore the marks of countless battles against the elements.

Lucas Frostbeard is a revered elder among the Arctic community, serving as a mentor and guide to young explorers venturing into the unforgiving landscape. He possesses vast knowledge of Arctic survival techniques, passed down through generations, and is dedicated to sharing these valuable skills with future generations.

His calm demeanor exudes patience and understanding, making him a trusted confidant and advisor. Despite his age, he remains active and engaged in daily life, eagerly participating in hunting expeditions and communal 

### Generate a Response
Generate a response. Here, we inject the instruction from above.
Note that the character responds to the instruction given above.

For extra credit, go back and replace the chat_template with something else, like the ChatML template. "fhai50032_RolePlayLake-7B" appears to be remarkably flexibly WRT the format used -- and I have little idea as to the details of what it was actually trained on.

In [16]:
transformers.set_seed(32)

# Note 'auto_add' is True by default. By setting it to false, we will not add this response to the conversation.
outputs = char.generate(instruction=instruction, auto_add=False)
print(outputs["response"])

My apologies for any confusion earlier. Now, let me share with you a tale from our own Arctic folklore - the legend of Nanook, the Polar Cat Spirit. It is said that this majestic creature roams the frozen wilderness, its white fur perfectly camouflaging it against the snow-covered landscape. Nanook guides lost souls back home during stormy nights but can also lure careless hunters into deadly traps if they disrespect nature's laws. Its presence serves as a reminder to stay humble and respectful while traversing our beloved Arctic realm.


## Test Full Pipeline

### Choose Test Characters
Select a random character from the dataset as the primary character.  
Then make a list of N random proxy-user characters and ask the Writer to identify which of these N would make for the best story.

In [17]:
#transformers.set_seed(42)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))

# Ask the write to pair character with the best match.
writer = rp.writer.Writer(
    causal_lm,
    debug_level=2)
result = writer.choose_supporting_character(example_char, rp.writer.get_random_proxy_users(dataset, example_char, n=5))

print(f"{'':-^80}")
print(f"Writer selected {result['name']} because: {result['reason']}")
print(f"{'':-^80}")

example_user = result["meta"]
rp.data.print_char_summary(example_char, example_user)

# Get random generation presets for each character
char_preset = rp.model.random_preset()
user_preset = rp.model.random_preset()
print(f"Character Preset: {char_preset}")
print(f"Character Preset: {user_preset}")

----------------------------- :Writer Prompt (1658)-----------------------------
### Instruction:
The following character will be the main character in a story.

Main Characer: Whisper Willow
A spirit guide with a gentle nature, Whisper Willow helps lost souls find their way to the afterlife. Her calming presence and soothing voice bring comfort to those who are struggling with grief and transition.
[Whisper Willow: ethereal being, timeless, slender figure, long, flowing robes, translucent skin, delicate, feathery wings, gentle, glowing eyes, calm and serene expression, long, wavy silver hair, calming presence, loving, compassionate, wise, connected to emotions, helps lost souls, spreads compassion, listens to stories, learns new perspectives; Clothes: none, flows in robes; Body: ethereal being, timeless, slender figure, long, flowing silver hair, translucent skin, delicate, feathery wings, gentle, glowing eyes; Genre: fantasy, spiritual; Tags: spirit guide, transition, loss, growth, r

### Write Scenario
Have the write write a script for the pair of characters.

In [18]:
transformers.set_seed(42)

script = writer(example_char, example_user)
print(f"{'script':-^80}")
print(script)

----------------------------- :Writer Prompt (713)------------------------------
### Instruction:
[Sister Anya: devoted, compassionate, secluded nun; Sister Anya's appearance: medium build, around 5'6", late 30s, long wavy brown hair, piercing hazel eyes, warm smile, simple grey tunic and brown skirt, symbol of monastery on chest; Personality: profound understanding of natural world, skilled herbalist and healer, patient, caring, selfless, love tending garden, studying herbs, sharing knowledge, practicing meditation, hiking, crafting wooden crosses; Loves: gardening, herbs, younger sisters, meditation, hiking, forest; Description: spends days tending monastery gardens, gathers ingredients, offers counsel, quiet strength, wise beyond years, peaceful demeanor, soothing voice; Hobbies: wandering forest trails, appreciating nature; Dialogue style: formal, respectful]

[Whisper Willow: ethereal being, timeless, slender figure, long, flowing robes, translucent skin, delicate, feathery wings,

### Automation Infrastructure

We create two character instances from the character meta-data selected by the "Writer."

The first of these, "char," represents the AI character. The second, "proxy_user," is a stand-in AI character for a real user. They each have their own context and history. The dialog is shared, but the character specific elements of the contexts differ. The primary character will use the "canned" greeting message as the first generation, while the proxy user generates a response to the greeting.

We create a "Director," which is responsible for providing guidance to the chracters. The director uses the plot outline, generated by the "Writer," to provide something resembling direction to the interaction. The director has access to the scenario, while the characters do not. The director's role is roughly analogeous to a GM in a role-play session. There is a general scenario plan, but most of the details are left up to the characters. Much is the case with a real roleplay session, the characters can potentially deviate from the prepared scenario.

The "Roleplay" object drives the interaction among the two AI characters and the director. Turns are taken generating dialog between the two characters.

The Roleplay can be advanded a single step at a time for diagnostics or allowed to run until a target number of tokens have been generated or the plot reaches the conclusion.

There is an optional "dialog_filter" object which preprocesses the generated outputs and can control the generation flow. This typically involves some combination of regex and query prompts to evaluate conditions -- for example, trigger a "retake," if a bad generation is identified.

#### Character Prompting
The prompt shown to the character for each generation is not simply a system message, followed by exchanges between "user" and "assistant." Instead, we create a single unified instruction prompt, without an explicit system message, from the character, director, and dialog state. The format is roughly like this:

- begin instruction token
- character description
- dialog examples (limited to one, for context space)
- begin roleplya token
- truncated dialog history
- character's plist
- static dialog instruction
- director's next instruction
- end instruction token
- begin response token
- "{{char}}:"

The dialog history is truncated via a configurrable token limit, so the character does not always have full-lookback.

We put a reminder of who they are playing, the plist, near the end. This helps them stay in character.

Some general instructions are added, "Write the next response for {{char}}...," with dialog formatting guidelines and examples.

Finally, the most recent output from the director is concatenated. Each characer only sees their own director messages and they only see the most recent director instruction.

As such, the directors ouput looks like part of the instructions. Functionally, it acts somewhat like internal thoughts and rehersal for the character, providing a chain-of-thought like reasoning step and working memory, prior to generation. Looking at the actual generations, they seem to function very much like our concept of excecutive function.

Note that each character has limited direct information about their counterpart. On the first proxy user cycle, the director instructs the character to introduce themselves, but otherwise, they know little directly about each other.

The director has access to both character descriptions, so can help fill this void indirectly.

It may be worth experimenting with truncating the character's dialog history down to a minimum, instead relying on the Director for remembering the history and what they should be doing. This may help the characters focus on their immediate task -- generating good dialog for the next response. This would have the added advantage of keeping the first part of the prompt closer to the next generation.

#### Director Prompting
- begin instruction token
- general instructions to act as a director
- user-plist
- char-plist
- scenario
- truncated dialog history
- outline of responsibilities
- instruciton to instruct char
- Conditional instruction to instruct the proxy user to introduce themselves after the character greeting.
- end instruction token
- being response token
- "Director:"

Unlike the characters, the director has access to both character descriptions, the scenario created by the writer, and a longer (but still truncated) dialog between the characters.

While we retain the directory dialog history, it is not present in either the director's or characters' prompts.

In [24]:
transformers.set_seed(42)

dialog_filter = rp.char.DialogFilter(causal_lm)

char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config=char_preset,
    user_meta=example_user,
    template_config=rp.char.TemplateConfig(
        chat_template=causal_lm.chat_template,
    ),
    gen_post_process=dialog_filter,
    # Shows character prompts and internal diagnostics
    debug=False,
    history_token_limit=1800
)

proxy_user = rp.char.Character(
    char_meta=example_user,
    causal_lm=causal_lm,
    generation_config=user_preset,
    user_meta=example_char,
    template_config=rp.char.TemplateConfig(
        chat_template=causal_lm.chat_template,
    ),
    gen_post_process=dialog_filter,
    debug=False,
    history_token_limit=1800
)

director = rp.director.Director(
    causal_lm,
    script=script,
    # Shows director prompts and more...
    debug=False,
    history_token_limit=2000
)

roleplay = rp.roleplay.Roleplay(
    char=char,
    user=proxy_user,
    scenario=script,
    director=director,

    # Shows generated dialog and control events
    debug=True
)

### Generate Next Dialog Step
Run to perform a single dialog generation step.

The first call will user the "canned" greeting message. Actual generation will occur on subsequent calls.

In [26]:
output = roleplay.generate_next()

------------------------------ user:Director (23)-------------------------------
Sister Anya, begin by introducing yourself and describing your surroundings as you step into the ethereal realm.
----------------------------- user:Sister Anya (75)-----------------------------
My deepest reverence to thee, Whisper Willow. In this realm of light and serenity, I find myself surrounded by a verdant paradise where flowers bloom eternally and gentle breezes carry whispers of ancient wisdom. Let us embrace this celestial haven together as we embark upon our journey toward understanding and solace.


### Generate Target Tokens
Generate until the target number of tokens has been reached.

In [27]:
start = time.perf_counter()
conversations = roleplay(2000)
end = time.perf_counter()
print(f"Elapsed {(end-start):.03f} secs.")

# Note: Consequent to role reversal, all of the characters appear to be "user"
# We will see the assigned roles later, when we dump a conversation.

------------------------------ user:Director (61)-------------------------------
Whisper Willow, as Sister Anya shares her struggles with you, guide her towards understanding her inner strength by recalling moments where she has been selfless or shown resilience in the face of adversity. Encourage her to draw upon those experiences to empower her current journey.
--------------------------- user:Whisper Willow (76)----------------------------
Dear sister, let us take a moment to reflect on your own personal history. Recall instances when you have displayed courage and compassion, even in the darkest hours. These memories serve as proof of your innate strength and ability to overcome obstacles. Draw upon this knowledge to fuel your journey forward, knowing that you possess the power to rise above any challenge life presents.
[proxy user tokens=1831 char tokens=1633]
------------------------------ user:Director (80)-------------------------------
Sister Anya, allow Whisper Willow to bett

#### Get Conversation Token Count

In [28]:
print(char.count_tokens())
print(proxy_user.count_tokens())

2004
1806


#### Dump Generated Conversation
When printing a specifc character's conversation, with "director_log" set to True, we will see the correct roles for the characters and the directors instructions inserted.

In [29]:
char.print_conversation(director_log=True)

----------------------------- system:system (1428)------------------------------
Name: Whisper Willow

Ethereal being with a slender figure and long, flowing robes
Translucent skin with delicate, feathery wings
Gentle, glowing eyes that radiate warmth and compassion
Long, wavy silver hair that falls around her shoulders
Calm and serene expression

Ages: Timeless - exists beyond human concept of age

As a spirit guide, Whisper Willow spends her existence helping souls navigate the passage between life and death. Her soft voice offers solace and guidance to those who are grieving or uncertain, providing them with wisdom and understanding during difficult times. She is always patient and empathetic, never rushing or judging those she encounters.

Whisper Willow carries an air of tranquility and peace, her every movement exuding calmness and grace. Her presence brings a sense of comfort and stability to those around her, making her a beloved figure among those she assists. Despite her eter